In [32]:
!pip install tqdm

In [3]:
import FinanceDataReader as fdr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score

In [36]:
from tqdm import tqdm
import FinanceDataReader as fdr
import pandas as pd

for idx, i in enumerate(tqdm(fdr.StockListing("KOSDAQ")['Code'])):
    if idx == 150:
        break
    a = fdr.DataReader(i)

    selected_features = a[['Close', 'Volume']].copy()

    selected_features.loc[:, 'MA5'] = selected_features['Close'].rolling(window=5).mean()
    selected_features.loc[:, 'MA10'] = selected_features['Close'].rolling(window=10).mean()
    selected_features.loc[:, 'MA20'] = selected_features['Close'].rolling(window=20).mean()

    selected_features.loc[:, 'STD10'] = selected_features['Close'].rolling(window=10).std()
    selected_features.loc[:, 'STD20'] = selected_features['Close'].rolling(window=20).std()

    selected_features.loc[:, 'Price_Diff'] = selected_features['Close'].shift(1) - selected_features['Close']

    selected_features.loc[:, 'Volume_Diff'] = selected_features['Volume'].shift(1) - selected_features['Volume']

    selected_features.loc[:, 'Return'] = selected_features['Close'].pct_change()
    selected_features.loc[:, 'Volume_Change'] = selected_features['Volume'].pct_change()

    selected_features = selected_features.dropna()

  9%|██████▋                                                                        | 150/1758 [01:20<14:18,  1.87it/s]


In [ ]:
X = selected_features[['Close', 'Volume', 'MA10', 'MA20', 'Return', 'Volume_Change']]
y = selected_features['Close'].shift(-1) 

y = y.dropna()
X = X.loc[y.index]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


model = RandomForestRegressor(random_state=42)


params = {
    'n_estimators': [100, 300, 500],   
    'max_depth': [10, 30, 50],        
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],     
    'max_features': ['auto', 'sqrt', 'log2'] 
}

grid_search = GridSearchCV(model, param_grid=params, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print(f"Best Parameters: {grid_search.best_params_}")

In [ ]:
import numpy as np

y_pred = grid_search.predict(X_test)

y_test_binary = np.where(y_test > y_test.shift(1), 1, 0)
y_pred_binary = np.where(y_pred > y_test, 1, 0)


print(f"y_test_binary: {y_test_binary[:5]}")
print(f"y_pred_binary: {y_pred_binary[:5]}")

In [ ]:
from sklearn.metrics import precision_score

precision = precision_score(y_test_binary, y_pred_binary)
print(f"Precision: {precision}")